# Tracking Sensor Bias

We want to compute the joint posterior over sensors' biases in a 2-D tracking setting.

In [9]:
from collections import OrderedDict

import torch
from torch.optim import Adam

import pyro
import pyro.distributions as dist

import funsor
import funsor.pyro
import funsor.distributions as f_dist
import funsor.ops as ops
from funsor.pyro.convert import dist_to_funsor, mvn_to_funsor, matrix_and_mvn_to_funsor, tensor_to_funsor
from funsor.interpreter import interpretation, reinterpret
from funsor.optimizer import apply_optimizer
from funsor.terms import lazy
from funsor.domains import bint, reals
from funsor.sum_product import sequential_sum_product

import matplotlib.pyplot as plt

Simulate some synthetic data:

In [10]:
num_sensors = 5
num_frames = 100

# simulate biased sensors
sensors  = []
for _ in range(num_sensors):
    bias = 0.5 * torch.randn(2)
    sensors.append(bias)

# simulate a single track
track = []
z = 10 * torch.rand(2)  # initial state
v = 2 * torch.randn(2)  # velocity
for t in range(num_frames):
    # Advance latent state.
    z += v + 0.1 * torch.randn(2)
#     z.clamp_(min=0, max=10)  # keep in the box
    
    # Observe via a random sensor.
    sensor_id = pyro.sample('id', dist.Categorical(torch.ones(num_sensors)))
    x = z - sensors[sensor_id]
    track.append({"sensor_id": sensor_id, "x": x})

Now let's set up a tracking problem in Funsor. We start by modeling the biases of each sensor.

In [11]:
%pdb on

Automatic pdb calling has been turned ON


In [12]:
# TODO transform this to cholesky decomposition
# print(bias_cov.shape)
# bias_cov = bias_cov @ bias_cov.t()
# create a joint Gaussian over biases
covs = [torch.eye(2, requires_grad=True) for i in range(num_sensors)]
bias = 0.
for i in range(num_sensors):
    bias += funsor.pyro.convert.mvn_to_funsor(
        dist.MultivariateNormal(torch.zeros(2), covs[i]),
        event_dims=("pos",),
        real_inputs=OrderedDict([("bias_{}".format(i), reals(2))])
    )(value="bias_{}".format(i))
bias.__dict__

{'inputs': OrderedDict([('bias_0', reals(2,)),
              ('bias_1', reals(2,)),
              ('bias_2', reals(2,)),
              ('bias_3', reals(2,)),
              ('bias_4', reals(2,))]),
 'output': reals(),
 'fresh': frozenset(),
 'bound': frozenset(),
 'deltas': (),
 'discrete': Tensor(-9.189385414123535, OrderedDict(), 'real'),
 'gaussian': Gaussian(..., ((bias_0, reals(2,)), (bias_1, reals(2,)), (bias_2, reals(2,)), (bias_3, reals(2,)), (bias_4, reals(2,)),)),
 '_ast_values': ((),
  Tensor(-9.189385414123535, OrderedDict(), 'real'),
  Gaussian(..., ((bias_0, reals(2,)), (bias_1, reals(2,)), (bias_2, reals(2,)), (bias_3, reals(2,)), (bias_4, reals(2,)),)))}

In [13]:
# original
# bias = sum(
#     funsor.pyro.convert.mvn_to_funsor(
#         dist.MultivariateNormal(
#             torch.zeros(2),
#             torch.eye(2, requires_grad=True)  # This can be learned
#         )
#     )(value="bias_{}".format(i))
#     for i in range(num_sensors)
# )

Set up the filter in funsor.

In [14]:
from pdb import set_trace as bb

In [24]:
def model(track):
    init_dist = torch.distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
    # TODO
    # this can be parameterized by a lower dimensional vector 
    # to learn a structured transition matrix
    # eg a GP with a matern v=3/2 kernel
    # see paper for details 
    transition_matrix = torch.randn(2, 2, requires_grad=True)

    transition_dist = torch.distributions.MultivariateNormal(
        torch.zeros(2),
        torch.eye(2))
    observation_matrix = torch.eye(2) + 0.2 * torch.randn(2, 2)
    observation_dist = torch.distributions.MultivariateNormal(
        torch.zeros(2),
        torch.eye(2))

    init = dist_to_funsor(init_dist)(value="state")
    # inputs are the previous state ``state`` and the next state
    trans = matrix_and_mvn_to_funsor(transition_matrix, transition_dist,
                                     ("time",), "state", "state(time=1)")
    obs = matrix_and_mvn_to_funsor(observation_matrix, observation_dist,
                                   ("time",), "state(time=1)", "value")
    
    # Now this is the crux, we add bias to the observation
    sensor_ids = funsor.torch.Tensor(
        torch.tensor([frame["sensor_id"] for frame in track]),
        OrderedDict([("sensor_id", bint(num_frames))]),
        dtype=len(sensors)
    )
    biased_observations = funsor.torch.Tensor(
        torch.stack([frame["x"] for frame in track]),
        OrderedDict([("value", bint(num_frames))])
    )
    bias_over_time = bias(sensor_id=sensor_ids)
    obs = obs(value=biased_observations)
    # Similar to funsor.pyro.hmm.GaussianHMM.log_prob()
    # ndims = max(len(batch_shape), value.dim() - event_dim)
    # value = tensor_to_funsor(value, ("time",), event_output=event_dim - 1,
    #                          dtype=self.dtype)

    # obs = obs(value=value)
    result = trans + obs
    bb()

    result = sequential_sum_product(ops.logaddexp, ops.add,
                                    result, "time", {"state": "state(time=1)"})
    result += init
    result = result.reduce(ops.logaddexp, frozenset(["state", "state(time=1)"]))
    import pdb; pdb.set_trace()
    # ensure we collapsed out the right dim
    assert result.data.dim() == 0
    return result

## Inference

Finally we have a result that is a joint Gaussian over the biases.
We can
1. optimize all parameters to maximize `result.reduce(obs.logaddexp)`
2. estimate the joint distribution over all bias parameters.

In [25]:
num_epochs = 200
# params = [bias_cov, transition_matrix]
params = covs
optim = Adam(params, lr=1e-3)
for i in range(num_epochs):
    optim.zero_grad()
    with interpretation(lazy):
        log_prob = apply_optimizer(model(track))
    loss = -reinterpret(log_prob).data
    loss.backward()
    if i % 10 == 0:
        print(loss)
    optim.step()
print(params)

> <ipython-input-24-1e9bee2b04b4>(47)model()
-> result = sequential_sum_product(ops.logaddexp, ops.add,
(Pdb) obs
(tensor(-1.8379) + Binary(add, Gaussian(tensor([[  0.1201,   0.1777],
        [  0.6589,  -0.7249],
        [  1.3408,  -1.5653],
        [  1.7657,  -2.4050],
        [  2.6114,  -3.7786],
        [  2.6402,  -4.1607],
        [  4.0417,  -6.0849],
        [  4.9014,  -6.8800],
        [  4.3693,  -6.9827],
        [  4.9077,  -7.8910],
        [  6.6962,  -9.7206],
        [  5.9895,  -9.7265],
        [  7.7263, -11.5969],
        [  7.2662, -11.5610],
        [  8.2741, -13.0112],
        [  8.2353, -13.3633],
        [  9.9671, -15.1411],
        [  9.3980, -15.0463],
        [ 10.0467, -16.0316],
        [ 11.1821, -17.4374],
        [ 11.9551, -18.8577],
        [ 12.2221, -19.1618],
        [ 12.8748, -20.1079],
        [ 13.8583, -21.3181],
        [ 13.3654, -21.3223],
        [ 13.8316, -22.1995],
        [ 15.2072, -24.0708],
        [ 15.9629, -24.7447],
      

(Pdb) qiot
*** NameError: name 'qiot' is not defined
(Pdb) quit


BdbQuit: 

> /Users/jpchen/anaconda/envs/pyro3/lib/python3.6/bdb.py(70)dispatch_line()
     68         if self.stop_here(frame) or self.break_here(frame):
     69             self.user_line(frame)
---> 70             if self.quitting: raise BdbQuit
     71         return self.trace_dispatch
     72 

ipdb> quit


Visualize the joint posterior distribution.